In [25]:
from src.constants import *
import pandas as pd
from constants import SPACY_DIR, ARTIFACTS_DIR
import spacy
from spacy.vocab import Vocab
from collections import Counter
import tqdm.notebook as tqdm
import os

In [2]:
abstracts = pd.read_pickle(JOURNALS_DF)[ABSTRACTS]

In [18]:
framing_map = dict()
adj_set = Counter()
noun_set = Counter()

missing = 0

with tqdm.tqdm(abstracts.index) as bar:
    for ident in bar:
        path = join(SPACY_DIR, f"{ident}.spacy")
        if not os.path.exists(path):
            missing =+ 1
            bar.set_postfix({"Missing": missing})
            continue

        doc = spacy.tokens.doc.Doc(Vocab()).from_disk(path)


        for token in doc:
            if token.pos_ == "NOUN":
                noun = str(token.text)
                noun_set.update([noun])

                if noun not in framing_map:
                    framing_map[noun] = Counter()

                adjectives = []

                for child in token.children:
                    if child.pos_ == "ADJ":
                        adjective = str(child.text)
                        framing_map[noun].update([adjective])
                        adj_set.update([adjective])


In [19]:
print(noun_set.most_common(10))
print(adj_set.most_common(10))

[('work', 23772), ('study', 13562), ('practice', 12088), ('research', 12025), ('care', 11244), ('workers', 11037), ('article', 10165), ('children', 10082), ('health', 9835), ('child', 7644)]
[('social', 39064), ('mental', 4080), ('other', 3642), ('professional', 3132), ('older', 2845), ('significant', 2578), ('qualitative', 2541), ('young', 2447), ('new', 2358), ('current', 2114)]


In [28]:
from collections import OrderedDict

nouns = set([n for n, _ in noun_set.most_common(1000)])
adjectives = set([a for a, _ in adj_set.most_common(1000)])

all_values = nouns.union(adjectives)

print("Creating dict")
occurrences = OrderedDict((str(key), OrderedDict((str(key), 0) for key in all_values)) for key in all_values)

print("Filling map")
# Find the co-occurrences:
for noun, counter in framing_map.items():
    if noun in nouns:
        for adj, count in counter.items():
            if adj in adjectives:
                occurrences[noun][adj] += count

Creating dict
Filling map


In [29]:
import pandas as pd
import csv

df = pd.DataFrame(occurrences)
df.to_csv(join(ARTIFACTS_DIR, "framing.csv"), sep=";", quoting=csv.QUOTE_ALL)